In [38]:
from vanna.openai import OpenAI_Chat
from vanna.pgvector import PG_VectorStore

from langchain_openai import OpenAIEmbeddings

In [39]:
import os
from dotenv import load_dotenv

load_dotenv()

class DalgoVannaClient(PG_VectorStore, OpenAI_Chat):
    def __init__(self, openai_config={}, pg_vector_config={}):
        PG_VectorStore.__init__(
            self,
            config={
                "connection_string": "postgresql+psycopg://{username}:{password}@{server}:{port}/{database}".format(
                    **{
                        "username": os.environ["PGVECTOR_USER"],
                        "password": os.environ["PGVECTOR_PASSWORD"],
                        "server": os.environ["PGVECTOR_HOST"],
                        "port": os.environ["PGVECTOR_PORT"],
                        "database": os.environ["PGVECTOR_DB"],
                    }
                ),
                **pg_vector_config,
            },
        )
        OpenAI_Chat.__init__(
            self,
            config={
                "api_key": os.environ["OPENAI_API_KEY"],
                "model": "gpt-4o-mini",
                **openai_config,
            },
        )


In [40]:
vn_client = DalgoVannaClient(
    openai_config={
        "initial_prompt": "Please qualify all table names with their schema names in the generated SQL"
    },
    pg_vector_config={"embedding_function": OpenAIEmbeddings()},
)

In [41]:
vn_client.connect_to_postgres(
    host=os.environ["WAREHOUSE_HOST"],
    dbname=os.environ["WAREHOUSE_DBNAME"],
    user=os.environ["WAREHOUSE_USER"],
    password=os.environ["WAREHOUSE_PASSWORD"],
    port=os.environ["WAREHOUSE_PORT"]
)

In [42]:
# traning plan
exclude_schemas = ["airbyte_internal", "dbt_staging_elementary", "pg_catalog"]

quote_schema = lambda schema: f"'{schema}'"

training_sql_query = f"""SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE \
table_schema not in (\
{','.join([quote_schema(schema) for schema in exclude_schemas])}\
) """

train_df = vn_client.run_sql(training_sql_query)

In [43]:
training_sql_query

"SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE table_schema not in ('airbyte_internal','dbt_staging_elementary','pg_catalog') "

In [54]:
train_df["table_schema"].value_counts()

table_schema
information_schema    649
mongo_staging         291
test_ledger           133
compliance            117
dbt_staging            52
Name: count, dtype: int64

In [53]:
training_plan = vn_client.get_training_plan_generic(df=train_df)

In [55]:
vn_client.train(plan=training_plan)

In [56]:
import sqlparse

sql = """
SELECT sum(total_ulbs) AS \"SUM(total_ulbs)\"\nFROM mongo_staging.ulbs_in_state\nLIMIT 50000;\n\n
"""
formatted_sql = sqlparse.format(sql, reindent=True, keyword_case='upper')
vn_client.train(question="What is the total count of ulbs ?", sql=formatted_sql)

'24a4646e-733d-43cd-819a-d65908f2df69-sql'

In [57]:
sql = """
SELECT sum(approved_ulbs) AS "SUM(approved_ulbs)"
FROM mongo_staging.tax_sub_rate
LIMIT 50000;
"""
vn_client.train(question="What is the total count of approved ulbs ?", sql=sql)

'c4fbb58a-4968-4de8-b7ab-96301d2a50ae-sql'

In [58]:
sql = """
SELECT SUM(approved_ulbs)/SUM(total_ulbs)*100 AS "SUM(approved_ulbs)/SUM(total_ulbs)*100"
FROM mongo_staging.tax_sub_rate
WHERE year IN ('2023-24')
LIMIT 50000;
"""
vn_client.train(question="What is the rate of tax submission acrosss ulbs ?", sql=sql)

'659ca155-77de-4dca-b907-0b3636fbf3ec-sql'

In [59]:
sql ="""
SELECT state AS "State",
       ulb AS "ULB",
       is_eligible AS "Eligible for Property tax Condition"
FROM mongo_staging.grants_available
GROUP BY state,
         ulb,
         is_eligible
LIMIT 1000;
"""
vn_client.train(question="How many cities (ulbs) are eligible ?", sql=sql)

'8ef1c25c-2a21-4585-8a9c-1ba0a178e0d3-sql'

In [60]:
sql = """
SELECT is_eligible AS is_eligible,
       count(is_eligible) AS "COUNT(is_eligible)"
FROM mongo_staging.grants_available
GROUP BY is_eligible
ORDER BY "COUNT(is_eligible)" DESC
LIMIT 100;
"""
vn_client.train(question="What is the distribution of cities that are eligible vs not eligible ?", sql=sql)

'e5453ffb-c29d-4da1-8b57-32b05e149c27-sql'

In [61]:
sql = """
SELECT ulb AS ulb,
       sum(growth_rate) AS "Growth rate"
FROM mongo_staging.growth_rate
WHERE growth_rate < 2542.8571428571427
GROUP BY ulb
ORDER BY "Growth rate" DESC
LIMIT 50;
"""
vn_client.train(question="Give me the distribution of growth rate across ulbs ?", sql=sql)


'fc6dccf5-2753-48cb-bb11-318a005dabe8-sql'

In [62]:
sql = """
SELECT (SUM(value_2022_23)-SUM(value_2021_22))/SUM(value_2021_22)*100 AS "(SUM(value_2022_23)-SUM(value_2021_22))/SUM(value_2021_22)*100"
FROM mongo_staging.growth_rate
LIMIT 50000;
"""
vn_client.train(question="What is the year on year growth rate of all ulbs combined ?", sql=sql)


'3ecbd51d-03a8-44bf-9c3b-267d8d50fd74-sql'

In [63]:
sql = """
SELECT sum(current_growth_rate) AS "SUM(current_growth_rate)"
FROM mongo_staging.gsdp_rate
LIMIT 50000;
"""
vn_client.train(question="What is the total gsdp rate ?", sql=sql)

'6bb156e9-4d44-4433-8903-7a74c716bc5f-sql'

### Janaagraha

In [67]:
chart = [{"chart_name": "City Eligibility", "schema": "mongo_staging", "table": "grants_available"}]

# user_question = "Give me the % eligibility across panchayats, municipalities and area councils"

# user_question = "Give me the % eligibility of states"

# user_question = "Give me the % eligibility of panchayats"

user_question = "Give me the % eligibility of ulb's"

vanna_query = f"""
User's Question: {user_question}

Additional Context:
- Chart Title: {chart[0]['chart_name']}
- Schema: {chart[0]['schema']}
- Table: {chart[0]['table']}

Please provide a detailed answer considering the above context.
"""

sql = vn_client.generate_sql(question=vanna_query, allow_llm_to_see_data=False)

SQL Prompt: [{'role': 'system', 'content': "Please qualify all table names with their schema names in the generated SQL\n===Additional Context \n\nThe following columns are in the eligibility_state_percent table in the cityfinance database:\n\n|     | table_catalog   | table_schema   | table_name                | column_name            | data_type         |\n|----:|:----------------|:---------------|:--------------------------|:-----------------------|:------------------|\n| 337 | cityfinance     | mongo_staging  | eligibility_state_percent | eligibility_percentage | numeric           |\n| 338 | cityfinance     | mongo_staging  | eligibility_state_percent | total_cities           | bigint            |\n| 339 | cityfinance     | mongo_staging  | eligibility_state_percent | eligible_cities        | bigint            |\n| 565 | cityfinance     | mongo_staging  | eligibility_state_percent | state                  | character varying |\n| 566 | cityfinance     | mongo_staging  | eligibility

| Question | Answer | Sql |
|----------|----------|----------|
| Give me the % eligibility across panchayats, municipalities and area councils | Incorrect , its trying to compute % for state but that is also incorrect | Row 1, Col 3 |
| Give me the % eligibility of states | It got this almost right, in the sense it didn't do DISTINCT and so i was seeing a state & its percentage multiple times | Row 2, Col 3 |
| Give me the % eligibility of panchayats | Incorrect, it doesn't understand what panchayats are which is reasonable because the schema doesn't have that information explicitly | Row 3, Col 3 |
| Give me the % eligibility of ulb's | Correct | Row 3, Col 3 |

In [68]:
chart = [{"chart_name": "City Eligibility", "schema": "mongo_staging", "table": "grants_available"}]

# user_question = "Give me the % eligibility across panchayats, municipalities and area councils"

# user_question = "Give me the % eligibility of states"

user_question = "Give me the % eligibility of panchayats"

# user_question = "Give me the % eligibility of ulb's"

vanna_query = f"""
User's Question: {user_question}

Additional Context:
- Chart Title: {chart[0]['chart_name']}
- Schema: {chart[0]['schema']}
- Table: {chart[0]['table']}

Please provide a detailed answer considering the above context.
"""

sql = vn_client.generate_sql(question=vanna_query, allow_llm_to_see_data=False)

SQL Prompt: [{'role': 'system', 'content': "Please qualify all table names with their schema names in the generated SQL\n===Additional Context \n\nThe following columns are in the eligibility_state_percent table in the cityfinance database:\n\n|     | table_catalog   | table_schema   | table_name                | column_name            | data_type         |\n|----:|:----------------|:---------------|:--------------------------|:-----------------------|:------------------|\n| 337 | cityfinance     | mongo_staging  | eligibility_state_percent | eligibility_percentage | numeric           |\n| 338 | cityfinance     | mongo_staging  | eligibility_state_percent | total_cities           | bigint            |\n| 339 | cityfinance     | mongo_staging  | eligibility_state_percent | eligible_cities        | bigint            |\n| 565 | cityfinance     | mongo_staging  | eligibility_state_percent | state                  | character varying |\n| 566 | cityfinance     | mongo_staging  | eligibility

In [69]:
chart = [{"chart_name": "City Eligibility", "schema": "mongo_staging", "table": "grants_available"}]

# user_question = "Give me the % eligibility across panchayats, municipalities and area councils"

user_question = "Give me the % eligibility of states"

# user_question = "Give me the % eligibility of panchayats"

# user_question = "Give me the % eligibility of ulb's"

vanna_query = f"""
User's Question: {user_question}

Additional Context:
- Chart Title: {chart[0]['chart_name']}
- Schema: {chart[0]['schema']}
- Table: {chart[0]['table']}

Please provide a detailed answer considering the above context.
"""

sql = vn_client.generate_sql(question=vanna_query, allow_llm_to_see_data=False)

SQL Prompt: [{'role': 'system', 'content': "Please qualify all table names with their schema names in the generated SQL\n===Additional Context \n\nThe following columns are in the eligibility_state_percent table in the cityfinance database:\n\n|     | table_catalog   | table_schema   | table_name                | column_name            | data_type         |\n|----:|:----------------|:---------------|:--------------------------|:-----------------------|:------------------|\n| 337 | cityfinance     | mongo_staging  | eligibility_state_percent | eligibility_percentage | numeric           |\n| 338 | cityfinance     | mongo_staging  | eligibility_state_percent | total_cities           | bigint            |\n| 339 | cityfinance     | mongo_staging  | eligibility_state_percent | eligible_cities        | bigint            |\n| 565 | cityfinance     | mongo_staging  | eligibility_state_percent | state                  | character varying |\n| 566 | cityfinance     | mongo_staging  | eligibility

In [70]:
chart = [{"chart_name": "City Eligibility", "schema": "mongo_staging", "table": "grants_available"}]

user_question = "Give me the % eligibility across panchayats, municipalities and area councils"

# user_question = "Give me the % eligibility of states"

# user_question = "Give me the % eligibility of panchayats"

# user_question = "Give me the % eligibility of ulb's"

vanna_query = f"""
User's Question: {user_question}

Additional Context:
- Chart Title: {chart[0]['chart_name']}
- Schema: {chart[0]['schema']}
- Table: {chart[0]['table']}

Please provide a detailed answer considering the above context.
"""

sql = vn_client.generate_sql(question=vanna_query, allow_llm_to_see_data=False)

SQL Prompt: [{'role': 'system', 'content': "Please qualify all table names with their schema names in the generated SQL\n===Additional Context \n\nThe following columns are in the eligibility_state_percent table in the cityfinance database:\n\n|     | table_catalog   | table_schema   | table_name                | column_name            | data_type         |\n|----:|:----------------|:---------------|:--------------------------|:-----------------------|:------------------|\n| 337 | cityfinance     | mongo_staging  | eligibility_state_percent | eligibility_percentage | numeric           |\n| 338 | cityfinance     | mongo_staging  | eligibility_state_percent | total_cities           | bigint            |\n| 339 | cityfinance     | mongo_staging  | eligibility_state_percent | eligible_cities        | bigint            |\n| 565 | cityfinance     | mongo_staging  | eligibility_state_percent | state                  | character varying |\n| 566 | cityfinance     | mongo_staging  | eligibility

### Inrem